In [17]:
def ldistance(lst_a, lst_b):
    if len(lst_a) == 0:
        score = len(lst_b)
        aligned = list(zip([None] * len(lst_b), lst_b))
        return (score, aligned)
    elif len(lst_b) == 0:
        score = len(lst_a)
        aligned = list(zip(lst_a, [None] * len(lst_a)))
        return (score, aligned)
    elif lst_a[0] == lst_b[0]:
        (score, rec) = ldistance(lst_a[1:], lst_b[1:])
        return (score, [(lst_a[0], lst_b[0])] + rec)
    else:
        (rec_score_1, rec_1) = ldistance(lst_a, lst_b[1:])
        score_1 = 1 + rec_score_1
        (rec_score_2, rec_2) = ldistance(lst_a[1:], lst_b)
        score_2 = 1 + rec_score_2
        if score_1 <= score_2:
            score = score_1
            aligned = [(None, lst_b[0])] + rec_1
            return (score, aligned)
        else:
            score = score_2
            aligned = [(lst_a[0], None)] + rec_2
            return (score, aligned)
    

In [5]:
ldistance(['hello','foo','world'],['hello','bar','world'])

2

In [4]:
ldistance(['hello','world'],['hello','world'])

0

In [20]:
ldistance(['hello','foo','bar','baz','world'],['hello','bar','world'])

(2,
 [('hello', 'hello'),
  ('foo', None),
  ('bar', 'bar'),
  ('baz', None),
  ('world', 'world')])

In [10]:
ldistance([], ['foo','bar'])

(2, [(None, 'foo'), (None, 'bar')])

In [96]:
def levenshtein_alignment(lst_a, lst_b, key=None):
    # Create the memoization table
    table = [[None for j in range(len(lst_b) + 1)] for i in range(len(lst_a) + 1)]
    
    def rec_ldistance(idx_a, idx_b):
        nonlocal table
        if table[idx_a][idx_b] is not None:
            # Check if we've already memoized
            return table[idx_a][idx_b]
        else:
            len_a = len(lst_a) - idx_a
            len_b = len(lst_b) - idx_b
            if len_a == 0:
                score = len_b
                aligned = list(zip([None] * len_b, lst_b[idx_b:]))
                ret = (score, aligned)
            elif len_b == 0:
                score = len_a
                aligned = list(zip(lst_a[idx_a:], [None] * len_b))
                ret = (score, aligned)
            elif ((key is None and lst_a[idx_a] == lst_b[idx_b]) or
                  (key is not None and key(lst_a[idx_a]) == key(lst_b[idx_b]))):
                (score, rec) = rec_ldistance(idx_a + 1, idx_b + 1)
                ret = (score, [(lst_a[idx_a], lst_b[idx_b])] + rec)
            else:
                (rec_score_1, rec_1) = rec_ldistance(idx_a, idx_b + 1)
                score_1 = 1 + rec_score_1
                (rec_score_2, rec_2) = rec_ldistance(idx_a + 1, idx_b)
                score_2 = 1 + rec_score_2
                (rec_score_3, rec_3) = rec_ldistance(idx_a + 1, idx_b + 1)
                score_3 = 1 + rec_score_3
                def argmin(a):
                    return min(range(len(a)), key=lambda x : a[x])
                min_choice = argmin([score_1, score_2, score_3])
                if min_choice == 0:
                    score = score_1
                    aligned = [(None, lst_b[idx_b])] + rec_1
                    ret = (score, aligned)
                elif min_choice == 1:
                    score = score_2
                    aligned = [(lst_a[idx_a], None)] + rec_2
                    ret = (score, aligned)
                elif min_choice == 2:
                    score = score_3
                    aligned = [(lst_a[idx_a], None), (None, lst_b[idx_b])] + rec_3
                    ret = (score, aligned)
            # Save into the memoization table
            table[idx_a][idx_b] = ret
            return ret

    (score, alignment) = rec_ldistance(0, 0)
    return alignment

In [97]:
levenshtein_alignment(['hello','foo','qux','bar','baz','world'],['hello','bar','gotcha','world'])

[('hello', 'hello'),
 ('foo', None),
 ('qux', None),
 ('bar', 'bar'),
 ('baz', None),
 (None, 'gotcha'),
 ('world', 'world')]

In [98]:
def test_levenshtein_alignment():
    alignment = levenshtein_alignment(['hello', 'foo', 'qux', 'bar', 'baz', 'world'], ['hello', 'bar', 'gotcha', 'world'])
    assert(alignment[0] == ('hello', 'hello'))
    assert(alignment[1] == ('foo', None))
    assert(alignment[2] == ('qux', None))
    assert(alignment[3] == ('bar', 'bar'))
    assert(alignment[4] == ('baz', None))
    assert(alignment[5] == (None, 'gotcha'))
    assert(alignment[6] == ('world', 'world'))

    alignment = levenshtein_alignment(['hello', 'hello', 'hello', 'world'], ['foo', 'foo', 'foo', 'world'])
    assert(len(alignment) == 7)

In [99]:
test_levenshtein_alignment()

In [100]:
levenshtein_alignment(['hello','hello','hello'],['world','world','world','hello'])

[(None, 'world'),
 ('hello', None),
 (None, 'world'),
 ('hello', None),
 (None, 'world'),
 ('hello', 'hello')]

In [101]:
def argmin(a):
    return min(range(len(a)), key=lambda x : a[x])

argmin([9,6,7])

1